# Spam Classification Using Feature Engineering Method
Wan Nazirul Hafeez, March 2018
### My goal is to predict if a new sms is spam or non-spam. I assume that is much worse misclassify non-spam than misclassify an spam. (I don't want to have false positives)

The reason is because I normally don't check the spam messages.
The two possible situations are:
New spam sms in my inbox. (False negative).
OUTCOME: I delete it.
New non-spam sms in my spam folder (False positive).
OUTCOME: I probably don't read it.

I prefer the first option!!!

Type II error!

http://sanjaymeena.io/tech/spam_deep_learning/

# 1. Import libraries

In [1]:
import pandas as pd
import numpy as np

# 2. Import Data

In [23]:
#Lesson 3: Load Data From CSV
data = pd.read_csv('E:\Git\_DL\smsORI2.csv')
data.head(5)

,Class,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
data.Class.value_counts()

ham     4827
spam     747
Name: Class, dtype: int64

In [25]:
pd.DataFrame(data=data.sum().isnull(), columns=["Has null?"])

,Has null?
Class,False
SMS,False


# Neural Network

## Natural Language Processing

### A. Building Vectorize Classifier TFIDF

Now that you have your training and testing data, you can build your classifiers. To get a good idea if the words and tokens in the articles had a significant impact on whether the news was fake or real, you begin by using CountVectorizer and TfidfVectorizer.

In [26]:
y = [1 if tmp_y=='spam' else 0 for tmp_y in data.Class]
print(y[:10])

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]


In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the `max attributes` 
tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_features=1000) 

# Fit and transform the training data 
tfidf_mat = tfidf_vectorizer.fit_transform(data.SMS).toarray()

In [118]:
print(type(tfidf_mat),tfidf_mat.shape)

<class 'numpy.ndarray'> (5574, 1000)


In [119]:
tfidf_df = pd.DataFrame(tfidf_mat, columns=tfidf_vectorizer.get_feature_names())

In [120]:
tfidf_df.head(10)

,00,000,02,03,04,06,0800,08000839402,08000930705,10,...,years,yep,yes,yesterday,yo,yr,yrs,yup,ã¼,ãœ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
# Make training and test sets 
from sklearn.model_selection import train_test_split

#random reproducability
rs = 111
X_train, X_test, y_train, y_test = train_test_split(tfidf_mat, y, 
                                                    test_size=0.2, train_size=0.8, random_state=rs)

#### NN

In [122]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from keras import metrics

In [135]:
num_max = 1000

def get_simple_model():
    model = Sequential()
    model.add(Dense(201, activation='relu', input_shape=(num_max,)))
    model.add(Dropout(0.2)) #prevent overfitting 
    model.add(Dense(99, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return model

In [151]:
def check_model(model,xtr,ytr,xts,yts):
    model.fit(xtr,ytr,batch_size=32,epochs=10,verbose=1,validation_split=0.3)
    print(' ')
    model.evaluate(xts,yts)
    ####
    from keras.models import load_model

    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    del model  # deletes the existing model


In [ ]:
m = get_simple_model()
check_model(m,X_train,y_train,X_test,y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_74 (Dense)             (None, 201)               201201    
_________________________________________________________________
dropout_53 (Dropout)         (None, 201)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 99)                19998     
_________________________________________________________________
dropout_54 (Dropout)         (None, 99)                0         
_________________________________________________________________
dense_76 (Dense)             (None, 1)                 100       
Total params: 221,299
Trainable params: 221,299
Non-trainable params: 0
_________________________________________________________________
Train on 3121 samples, validate on 1338 samples
Epoch 1/10
3121/3121 [==============================] - 3s 880us/step - loss: 0.3442 - a

https://www.youtube.com/watch?v=mrMozuo7ZQg
-- how to create confusion matrix

In [142]:
predictions = model.predict(xts, verbose=1)
model.predict_classes(predictions, verbose=1)

NameError: name 'model' is not defined

In [140]:
from keras.preprocessing.text import Tokenizer

text = np.array(['this is just some random, stupid text'])
print(text.shape)

tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(text)
pred = tok.texts_to_sequences(text)

print(pred)

model.predict(pred)

prediction = model.predict(np.array(tk.texts_to_sequences(text)))
print(prediction)

(1,)
[[1, 2, 3, 4, 5, 6, 7]]


NameError: name 'model' is not defined

In [ ]:
def get_cnn_model_v1():   
    model = Sequential()
    model.add(Embedding(1000,20,input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])
    return model